# Preprocessing

## Imports

In [65]:
import numpy as np
import pandas as pd
import posixpath
import pycountry
import geopandas as gpd
import json
import pyproj
from itertools import accumulate
import operator

from info_vis import PROJECT_PATH, DATA_PATH, PREPROCESSED_DATA_PATH

## Load Data

### Unicorn Startups

In [66]:
filename = "World_Wide_Unicorn_Startups.csv"
unicorn_startups_df = pd.read_csv(posixpath.join(DATA_PATH, filename))
unicorn_startups_df

,Company,Valuation,Date,Country,City,Industry,Investors,year,month,day
0,Bytedance,140.0,4/7/2017,China,Beijing,Artificial intelligence,"0 Sequoia Capital China, SIG Asia Investm...",2017,7,4
1,SpaceX,100.3,12/1/2012,United States,Hawthorne,Other,"0 Sequoia Capital China, SIG Asia Investm...",2012,1,12
2,Stripe,95.0,1/23/2014,United States,San Francisco,Fintech,"0 Sequoia Capital China, SIG Asia Investm...",2014,23,1
3,Klarna,45.6,12/12/2011,Sweden,Stockholm,Fintech,"0 Sequoia Capital China, SIG Asia Investm...",2011,12,12
4,Canva,40.0,1/8/2018,Australia,Surry Hills,Internet software & services,"0 Sequoia Capital China, SIG Asia Investm...",2018,8,1
...,...,...,...,...,...,...,...,...,...,...
931,YipitData,1.0,12/6/2021,United States,New York,Internet software & services,"0 Sequoia Capital China, SIG Asia Investm...",2021,6,12
932,Anyscale,1.0,12/7/2021,United States,Berkeley,Artificial Intelligence,"0 Sequoia Capital China, SIG Asia Investm...",2021,7,12
933,Iodine Software,1.0,12/1/2021,United States,Austin,Data management & analytics,"0 Sequoia Capital China, SIG Asia Investm...",2021,1,12
934,ReliaQuest,1.0,12/1/2021,United States,Tampa,Cybersecurity,"0 Sequoia Capital China, SIG Asia Investm...",2021,1,12


### Quality Of Life

In [67]:
filename = "QOL.csv"
qol_df = pd.read_csv(posixpath.join(DATA_PATH, filename))
qol_df

,Rank,Country,Quality of Life Index,Purchasing Power Index,Safety Index,Health Care Index,Cost of Living Index,Property Price to Income Ratio,Traffic Commute Time Index,Pollution Index,Climate Index,Year
0,1,Switzerland,190.8,111.0,78.7,74.5,131.7,8.4,28.7,20.1,80.0,2021
1,2,Denmark,190.0,94.7,73.3,80.0,91.7,6.7,28.7,20.4,81.8,2021
2,3,Netherlands,183.3,83.9,72.8,75.8,78.6,7.4,27.8,25.3,87.1,2021
3,4,Finland,182.8,89.1,73.0,76.4,77.5,8.6,29.0,11.9,56.6,2021
4,5,Austria,182.4,78.2,74.8,78.4,75.5,10.4,25.7,19.2,77.8,2021
...,...,...,...,...,...,...,...,...,...,...,...,...
498,82,Bangladesh,-5.4,33.4,35.8,43.3,39.2,10.9,58.0,93.6,-,2015
499,83,Egypt,-7.1,25.8,39.9,54.7,37.2,11.3,56.9,96.6,-,2015
500,84,Vietnam,-19.5,21.3,47.7,34.3,41.9,23.9,36.4,84.3,-,2015
501,85,Mongolia,-35.7,23.5,29.7,28.3,52.3,18.6,36.9,94.6,-,2015


### GDP

In [68]:
filename = "GDP.csv"
gdp_df = pd.read_csv(posixpath.join(DATA_PATH, filename))
gdp_df

,Country/Area,Year,Unit,"GDP, Per Capita GDP - US Dollars"
0,Afghanistan,2015,US$,552.722397
1,Afghanistan,2016,US$,525.188137
2,Afghanistan,2017,US$,533.339054
3,Afghanistan,2018,US$,513.194331
4,Afghanistan,2019,US$,511.736489
...,...,...,...,...
1689,Zimbabwe,2017,US$,1487.978497
1690,Zimbabwe,2018,US$,1572.738770
1691,Zimbabwe,2019,US$,1479.534798
1692,Zimbabwe,2020,US$,1395.305090


## Preprocess Data

### Unicorn Startups

#### Drop wanted columns

In [69]:
unicorn_startups_df = unicorn_startups_df.drop(columns=["Investors", "month", "day", "year"])
unicorn_startups_df

,Company,Valuation,Date,Country,City,Industry
0,Bytedance,140.0,4/7/2017,China,Beijing,Artificial intelligence
1,SpaceX,100.3,12/1/2012,United States,Hawthorne,Other
2,Stripe,95.0,1/23/2014,United States,San Francisco,Fintech
3,Klarna,45.6,12/12/2011,Sweden,Stockholm,Fintech
4,Canva,40.0,1/8/2018,Australia,Surry Hills,Internet software & services
...,...,...,...,...,...,...
931,YipitData,1.0,12/6/2021,United States,New York,Internet software & services
932,Anyscale,1.0,12/7/2021,United States,Berkeley,Artificial Intelligence
933,Iodine Software,1.0,12/1/2021,United States,Austin,Data management & analytics
934,ReliaQuest,1.0,12/1/2021,United States,Tampa,Cybersecurity


#### 1. Change date format into yyyy
#### 2. Change Date column name into Year
#### 3. Sort by (year, valuation), in ascending and descending orders respectivly

In [70]:
unicorn_startups_df["Date"] = unicorn_startups_df["Date"].apply(lambda x: int(x[-4:]))
unicorn_startups_df = unicorn_startups_df.rename(columns={'Date': 'Year'})
unicorn_startups_df = unicorn_startups_df.sort_values(by=["Year", "Valuation"], ascending=[True, False]).reset_index(drop=True)
unicorn_startups_df

,Company,Valuation,Year,Country,City,Industry
0,Veepee,1.38,2007,France,La Plaine Saint-Denis,E-commerce & direct-to-consumer
1,VANCL,3.00,2010,China,Beijing,E-commerce & direct-to-consumer
2,Klarna,45.60,2011,Sweden,Stockholm,Fintech
3,Vice Media,5.70,2011,United States,Brooklyn,Internet software & services
4,SpaceX,100.30,2012,United States,Hawthorne,Other
...,...,...,...,...,...,...
931,YipitData,1.00,2021,United States,New York,Internet software & services
932,Anyscale,1.00,2021,United States,Berkeley,Artificial Intelligence
933,Iodine Software,1.00,2021,United States,Austin,Data management & analytics
934,ReliaQuest,1.00,2021,United States,Tampa,Cybersecurity


In [71]:
set(unicorn_startups_df["Industry"])

{'Artificial Intelligence',
 'Artificial intelligence',
 'Auto & transportation',
 'Consumer & retail',
 'Cybersecurity',
 'Data management & analytics',
 'E-commerce & direct-to-consumer',
 'Edtech',
 'Fintech',
 'Finttech',
 'Hardware',
 'Health',
 'Internet software & services',
 'Mobile & telecommunications',
 'Other',
 'Supply chain, logistics, & delivery',
 'Travel'}

In [72]:
def standarize_industries(x):
    if x == "Artificial intelligence":
        return "Artificial Intelligence"
    elif x == "Auto & transportation":
        return "Auto & Transportation"
    elif x == "Consumer & retail":
        return "Consumer & Retail"
    elif x == "Data management & analytics":
        return "Data management & Analytics"
    elif x == "E-commerce & direct-to-consumer":
        return "E-commerce & Direct-To-Consumer"
    elif x == "Finttech":
        return "Fintech"
    elif x == "Internet software & services":
        return "Internet Software & Services"
    elif x == "Mobile & telecommunications":
        return "Mobile & Telecommunications"
    elif x == "Supply chain, logistics, & delivery":
        return "Supply chain, Logistics, & Delivery"   
    else:
        return x
unicorn_startups_df["Industry"] = unicorn_startups_df["Industry"].apply(lambda x: standarize_industries(x)) 
set(unicorn_startups_df["Industry"])

{'Artificial Intelligence',
 'Auto & Transportation',
 'Consumer & Retail',
 'Cybersecurity',
 'Data management & Analytics',
 'E-commerce & Direct-To-Consumer',
 'Edtech',
 'Fintech',
 'Hardware',
 'Health',
 'Internet Software & Services',
 'Mobile & Telecommunications',
 'Other',
 'Supply chain, Logistics, & Delivery',
 'Travel'}

#### Clean data
1. Fill missing data (singapore and Hong Kong's city column is missing, take the capital by default)

In [73]:
mask = unicorn_startups_df['City'].isna()
unicorn_startups_df.loc[mask, 'City'] = unicorn_startups_df.loc[mask, 'Country']
unicorn_startups_df.isna().sum()

Company      0
Valuation    0
Year         0
Country      0
City         0
Industry     0
dtype: int64

#### Extract year range and existing countries

In [74]:
unicorn_startups_years = set(unicorn_startups_df["Year"])
unicorn_startups_countries = set(unicorn_startups_df["Country"])
print(unicorn_startups_years)
print(unicorn_startups_countries)

{2016, 2017, 2018, 2019, 2020, 2021, 2007, 2010, 2011, 2012, 2013, 2014, 2015}
{'Norway', 'Thailand', 'United Arab Emirates', 'Germany', 'Canada', 'Philippines', 'Indonesia,', 'Japan', 'United States', 'South Korea', 'Switzerland', 'Ireland', 'Hong Kong', 'Sweden', 'France', 'Finland', 'Estonia', 'Belgium', 'Bermuda', 'India', 'Croatia', 'Netherlands', 'Lithuania', 'Singapore', 'Vietnam', 'Santa Clara', 'South Africa', 'Turkey', 'Nigeria', 'United Kingdom', 'Austria', 'Chile', 'Brazil', 'Indonesia', 'Luxembourg', 'Australia', 'Czech Republic', 'Denmark', 'Colombia', 'Senegal', 'Israel', 'Malaysia', 'China', 'United States,', 'Mexico', 'Argentina', 'Spain'}


### Quality of Life

#### Drop wanted columns

In [75]:
qol_df = qol_df.drop(columns=["Rank", "Safety Index", "Health Care Index", "Traffic Commute Time Index", "Pollution Index", "Climate Index"])
qol_df

,Country,Quality of Life Index,Purchasing Power Index,Cost of Living Index,Property Price to Income Ratio,Year
0,Switzerland,190.8,111.0,131.7,8.4,2021
1,Denmark,190.0,94.7,91.7,6.7,2021
2,Netherlands,183.3,83.9,78.6,7.4,2021
3,Finland,182.8,89.1,77.5,8.6,2021
4,Austria,182.4,78.2,75.5,10.4,2021
...,...,...,...,...,...,...
498,Bangladesh,-5.4,33.4,39.2,10.9,2015
499,Egypt,-7.1,25.8,37.2,11.3,2015
500,Vietnam,-19.5,21.3,41.9,23.9,2015
501,Mongolia,-35.7,23.5,52.3,18.6,2015


#### Sort by (year, country) in ascending order

In [76]:
qol_df = qol_df.sort_values(by=["Year", "Country"]).reset_index(drop=True)
qol_df

,Country,Quality of Life Index,Purchasing Power Index,Cost of Living Index,Property Price to Income Ratio,Year
0,Argentina,77.0,59.4,67.1,11.6,2015
1,Armenia,49.1,27.7,40.7,13.4,2015
2,Australia,180.8,110.4,99.3,7.1,2015
3,Austria,182.6,104.6,76.9,9.6,2015
4,Bahrain,84.1,59.7,56.2,7.9,2015
...,...,...,...,...,...,...
498,United Arab Emirates,156.0,85.7,61.7,4.9,2021
499,United Kingdom,159.0,82.6,71.0,9.6,2021
500,United States,167.0,102.6,71.9,4.0,2021
501,Uruguay,124.6,30.6,51.1,17.3,2021


#### Extract year range and existing countries

In [77]:
qol_years = set(qol_df["Year"])
qol_countries = set(qol_df["Country"])
print(qol_years)
print(qol_countries)

{2016, 2017, 2018, 2019, 2020, 2021, 2015}
{'Kazakhstan', 'United States', 'Saudi Arabia', 'Switzerland', 'Estonia', 'Cambodia', 'Mongolia', 'North Macedonia', 'Peru', 'Ukraine', 'Lithuania', 'South Africa', 'Egypt', 'Brazil', 'Denmark', 'Bangladesh', 'Ecuador', 'Mexico', 'Bolivia', 'Cyprus', 'Costa Rica', 'Thailand', 'Qatar', 'United Arab Emirates', 'Germany', 'Philippines', 'Greece', 'Georgia', 'Latvia', 'South Korea', 'Ireland', 'Taiwan', 'Finland', 'France', 'Moldova', 'India', 'Netherlands', 'Sri Lanka', 'Romania', 'Vietnam', 'Turkey', 'Nigeria', 'Austria', 'Dominican Republic', 'Bahrain', 'Czech Republic', 'Australia', 'Azerbaijan', 'Belarus', 'Italy', 'Kenya', 'Pakistan', 'Russia', 'Puerto Rico', 'Norway', 'Lebanon', 'Bulgaria', 'Japan', 'Serbia', 'Hong Kong', 'New Zealand', 'Sweden', 'Belgium', 'Morocco', 'Bosnia And Herzegovina', 'Slovenia', 'Turkmenistan', 'Chile', 'Iceland', 'Colombia', 'Israel', 'China', 'Jordan', 'Canada', 'Uruguay', 'Panama', 'Poland', 'Portugal', 'Venezu

### GDP

#### Drop unwanted columns

In [78]:
gdp_df = gdp_df.drop(columns=["Unit"])
gdp_df

,Country/Area,Year,"GDP, Per Capita GDP - US Dollars"
0,Afghanistan,2015,552.722397
1,Afghanistan,2016,525.188137
2,Afghanistan,2017,533.339054
3,Afghanistan,2018,513.194331
4,Afghanistan,2019,511.736489
...,...,...,...
1689,Zimbabwe,2017,1487.978497
1690,Zimbabwe,2018,1572.738770
1691,Zimbabwe,2019,1479.534798
1692,Zimbabwe,2020,1395.305090


In [79]:
gdp_df = gdp_df.rename(columns={'GDP, Per Capita GDP - US Dollars': 'GDP Per Capita', "Country/Area": "Country"})
gdp_df

,Country,Year,GDP Per Capita
0,Afghanistan,2015,552.722397
1,Afghanistan,2016,525.188137
2,Afghanistan,2017,533.339054
3,Afghanistan,2018,513.194331
4,Afghanistan,2019,511.736489
...,...,...,...
1689,Zimbabwe,2017,1487.978497
1690,Zimbabwe,2018,1572.738770
1691,Zimbabwe,2019,1479.534798
1692,Zimbabwe,2020,1395.305090


#### Sort by (year, Country) in ascending order

In [80]:
gdp_df = gdp_df.sort_values(by=["Year", "Country"]).reset_index(drop=True)
gdp_df

,Country,Year,GDP Per Capita
0,Afghanistan,2015,552.722397
1,Africa,2015,1983.750763
2,Albania,2015,3928.362400
3,Algeria,2015,4147.453064
4,Americas,2015,25903.772991
...,...,...,...
1689,Western Europe,2021,51910.529680
1690,World,2021,12313.588733
1691,Yemen,2021,243.188488
1692,Zambia,2021,1127.523151


#### Extract year range and existing countries

In [81]:
gdp_years = set(gdp_df["Year"])
gdp_countries = set(gdp_df["Country"])
print(gdp_years)
print(gdp_countries)

{2016, 2017, 2018, 2019, 2020, 2021, 2015}
{'Saint Kitts and Nevis', 'Barbados', 'Middle Africa', 'Northern America', 'Marshall Islands', 'Trinidad and Tobago', 'Republic of Korea', 'Central Asia', 'Benin', 'China, Macao Special Administrative Region', 'Belize', 'Kazakhstan', 'Sint Maarten (Dutch part)', 'United States', 'Saudi Arabia', 'Switzerland', 'Malawi', 'Cabo Verde', 'Estonia', 'Nauru', 'Mauritius', 'Curaçao', 'Bermuda', 'Cambodia', 'Mongolia', 'Caribbean', 'Liechtenstein', 'Peru', 'Ukraine', 'Mauritania', 'South America', 'Lithuania', 'Anguilla', 'South Africa', 'Aruba', 'Tajikistan', 'Rwanda', 'Dominica', 'Micronesia', 'Egypt', 'Brazil', 'Greenland', 'Melanesia', 'Myanmar', 'Saint Lucia', 'Tuvalu', 'Bolivia (Plurinational State of)', 'Botswana', 'Vanuatu', 'Denmark', 'Bangladesh', 'United Kingdom of Great Britain and Northern Ireland', 'Türkiye', 'Suriname', 'Ecuador', 'Haiti', 'Nicaragua', 'Senegal', 'Mexico', 'South Sudan', 'Cyprus', 'Kyrgyzstan', 'State of Palestine', 'Chi

## Find commun years and countries among the 3 datasets and reduce them accordingly

In [82]:
common_years = gdp_years.intersection(qol_years.intersection(unicorn_startups_years))
common_countries = gdp_countries.intersection(qol_countries.intersection(unicorn_startups_countries))

print(common_years)
print(common_countries)
print(len(common_years))
print(len(common_countries))

{2016, 2017, 2018, 2019, 2020, 2021, 2015}
{'Norway', 'Thailand', 'United Arab Emirates', 'Canada', 'Germany', 'Philippines', 'Japan', 'United States', 'Switzerland', 'Ireland', 'Sweden', 'France', 'Finland', 'Estonia', 'Belgium', 'India', 'Croatia', 'Netherlands', 'Lithuania', 'Singapore', 'South Africa', 'Nigeria', 'Austria', 'Chile', 'Brazil', 'Indonesia', 'Australia', 'Denmark', 'Colombia', 'Malaysia', 'Israel', 'Mexico', 'Argentina', 'Spain'}
7
34


In [83]:
unicorn_startups_df = unicorn_startups_df[unicorn_startups_df["Year"].isin(common_years) & unicorn_startups_df["Country"].isin(common_countries)].reset_index(drop=True)
qol_df = qol_df[qol_df["Year"].isin(common_years) & qol_df["Country"].isin(common_countries)].reset_index(drop=True)
gdp_df = gdp_df[gdp_df["Year"].isin(common_years) & gdp_df["Country"].isin(common_countries)].reset_index(drop=True)
print(set(unicorn_startups_df["Year"]))
print(set(qol_df["Year"]))
print(set(gdp_df["Year"]))
print(set(unicorn_startups_df["Country"]))
print(set(qol_df["Country"]))
print(set(gdp_df["Country"]))
print(len(set(unicorn_startups_df["Country"])))
print(len(set(qol_df["Country"])))
print(len(set(gdp_df["Country"])))

{2016, 2017, 2018, 2019, 2020, 2021, 2015}
{2016, 2017, 2018, 2019, 2020, 2021, 2015}
{2016, 2017, 2018, 2019, 2020, 2021, 2015}
{'Norway', 'Thailand', 'United Arab Emirates', 'Germany', 'Canada', 'Philippines', 'Japan', 'United States', 'Switzerland', 'Ireland', 'Sweden', 'France', 'Finland', 'Estonia', 'Belgium', 'India', 'Croatia', 'Netherlands', 'Lithuania', 'Singapore', 'South Africa', 'Nigeria', 'Austria', 'Chile', 'Brazil', 'Indonesia', 'Australia', 'Denmark', 'Colombia', 'Malaysia', 'Israel', 'Mexico', 'Argentina', 'Spain'}
{'Norway', 'Thailand', 'United Arab Emirates', 'Canada', 'Germany', 'Philippines', 'Japan', 'United States', 'Switzerland', 'Ireland', 'Sweden', 'Finland', 'France', 'Estonia', 'Belgium', 'India', 'Croatia', 'Netherlands', 'Lithuania', 'Singapore', 'South Africa', 'Nigeria', 'Austria', 'Chile', 'Brazil', 'Indonesia', 'Australia', 'Denmark', 'Colombia', 'Malaysia', 'Israel', 'Mexico', 'Argentina', 'Spain'}
{'Norway', 'Thailand', 'United Arab Emirates', 'Canad

## Merge GDP and Quality of Life data into one dataset
#### The Quality of Life dataset now contains the GDP per Capita data

In [84]:
qol_df["GDP Per Capita"] = gdp_df["GDP Per Capita"]
qol_df

,Country,Quality of Life Index,Purchasing Power Index,Cost of Living Index,Property Price to Income Ratio,Year,GDP Per Capita
0,Argentina,77.0,59.4,67.1,11.6,2015,14833.199680
1,Australia,180.8,110.4,99.3,7.1,2015,52009.802759
2,Austria,182.6,104.6,76.9,9.6,2015,43908.420277
3,Belgium,136.0,86.2,87.2,6.5,2015,40889.673570
4,Brazil,29.8,41.2,55.3,16.7,2015,8936.195589
...,...,...,...,...,...,...,...
228,Sweden,171.4,90.6,79.2,8.6,2021,93076.833587
229,Switzerland,190.8,111.0,131.7,8.4,2021,3484.385958
230,Thailand,100.3,31.4,49.3,22.2,2021,78270.600992
231,United Arab Emirates,156.0,85.7,61.7,4.9,2021,6830.894725


### Drop countries that are not present in all years

In [85]:
len(countries)

32

In [86]:
year_counts = qol_df.groupby("Country")["Year"].nunique()
countries = year_counts[year_counts == len(set(common_years))].index
qol_df = qol_df[qol_df["Country"].isin(countries)].reset_index(drop=True)
unicorn_startups_df = unicorn_startups_df[unicorn_startups_df["Country"].isin(countries)].reset_index(drop=True)
qol_df

,Country,Quality of Life Index,Purchasing Power Index,Cost of Living Index,Property Price to Income Ratio,Year,GDP Per Capita
0,Argentina,77.0,59.4,67.1,11.6,2015,14833.199680
1,Australia,180.8,110.4,99.3,7.1,2015,52009.802759
2,Austria,182.6,104.6,76.9,9.6,2015,43908.420277
3,Belgium,136.0,86.2,87.2,6.5,2015,40889.673570
4,Brazil,29.8,41.2,55.3,16.7,2015,8936.195589
...,...,...,...,...,...,...,...
219,Sweden,171.4,90.6,79.2,8.6,2021,93076.833587
220,Switzerland,190.8,111.0,131.7,8.4,2021,3484.385958
221,Thailand,100.3,31.4,49.3,22.2,2021,78270.600992
222,United Arab Emirates,156.0,85.7,61.7,4.9,2021,6830.894725


## Standardizing column and countries (USA) names

In [87]:
unicorn_startups_df.columns = unicorn_startups_df.columns.str.strip().str.replace(' ', '_').str.replace('-', '_')
qol_df.columns = qol_df.columns.str.strip().str.replace(' ', '_').str.replace('-', '_')

def normalize_country(name):
    name = str(name).strip()
    name = name.replace('U.S.', 'United States').replace('USA', 'United States')
    return name.title()

unicorn_startups_df['Country'] = unicorn_startups_df['Country'].apply(normalize_country)
qol_df['Country'] = qol_df['Country'].apply(normalize_country)

## Merge with the ISO3 countries data

In [88]:
def country_to_iso3(name):
    try: return pycountry.countries.lookup(name).alpha_3
    except: return None

unicorn_startups_df['ISO3'] = unicorn_startups_df['Country'].apply(country_to_iso3)
qol_df["ISO3"] = qol_df["Country"].apply(country_to_iso3)

## Aggregate data for unicorn startups per Country

In [89]:
unicorn_startups_df

,Company,Valuation,Year,Country,City,Industry,ISO3
0,Gusto,10.00,2015,United States,San Francisco,Fintech,USA
1,Tanium,9.00,2015,United States,Kirkland,Cybersecurity,USA
2,Zenefits,4.50,2015,United States,San Francisco,Fintech,USA
3,Thumbtack,3.20,2015,United States,San Francisco,E-commerce & Direct-To-Consumer,USA
4,Illumio,2.75,2015,United States,Sunnyvale,Cybersecurity,USA
...,...,...,...,...,...,...,...
676,YipitData,1.00,2021,United States,New York,Internet Software & Services,USA
677,Anyscale,1.00,2021,United States,Berkeley,Artificial Intelligence,USA
678,Iodine Software,1.00,2021,United States,Austin,Data management & Analytics,USA
679,ReliaQuest,1.00,2021,United States,Tampa,Cybersecurity,USA


In [90]:
unicorn_startups_country_df = unicorn_startups_df.groupby(["ISO3", "Country", "Year"], as_index=False).agg(N_Unicorns = ("ISO3", "size"), 
                                                                                                   Total_Val = ("Valuation", "sum"),
                                                                                                   Startups = ("Valuation",
                                                                                                               lambda s: dict(zip(unicorn_startups_df.loc[s.index, "Company"], s))), 
                                                                                                   Industries = ("Industry", list), 
                                                                                                   #cities = ("City", list)
                                                                                                  ).sort_values(by=["Country", "Year"])
unicorn_startups_country_df[["N_Unicorns_Cumulative", "Total_Val_Cumulative"]] = unicorn_startups_country_df.groupby('Country').agg(N_Unicorns = ("N_Unicorns", "cumsum"), Total_Val = ("Total_Val", "cumsum"))
unicorn_startups_country_df = unicorn_startups_country_df.sort_values(by=["Year", "Country"]).reset_index(drop=True)
unicorn_startups_country_df

,ISO3,Country,Year,N_Unicorns,Total_Val,Startups,Industries,N_Unicorns_Cumulative,Total_Val_Cumulative
0,COL,Colombia,2015,1,1.15,{'LifeMiles': 1.15},[Other],1,1.15
1,FRA,France,2015,1,2.00,{'BlaBlaCar': 2.0},[Auto & Transportation],1,2.00
2,USA,United States,2015,14,44.17,"{'Gusto': 10.0, 'Tanium': 9.0, 'Zenefits': 4.5...","[Fintech, Cybersecurity, Fintech, E-commerce &...",14,44.17
3,FRA,France,2016,1,1.10,{'OVH': 1.1},[Other],2,3.10
4,ISR,Israel,2016,1,1.50,{'Gett': 1.5},[Auto & Transportation],1,1.50
...,...,...,...,...,...,...,...,...,...
78,SWE,Sweden,2021,2,3.40,"{'KRY': 2.0, 'Epidemic Sound': 1.4}","[Health, Internet Software & Services]",3,12.48
79,CHE,Switzerland,2021,1,1.10,{'Nexthink': 1.1},[Data management & Analytics],4,5.60
80,THA,Thailand,2021,2,2.50,"{'Ascend Money': 1.5, 'Flash Express': 1.0}","[Fintech, Supply chain, Logistics, & Delivery]",2,2.50
81,ARE,United Arab Emirates,2021,1,1.00,{'Kitopi': 1.0},"[Supply chain, Logistics, & Delivery]",3,4.50


### Fill missing data in Unicorn Startups data

In [91]:
df = unicorn_startups_country_df.copy()

# 1) Determine the full list of years and countries
years     = df['Year'].sort_values().unique()
countries = df['Country'].unique()

# 2) Build a full MultiIndex of Country × Year
full_idx = pd.MultiIndex.from_product(
    [countries, years],
    names=['Country','Year']
)

# 3) Re‑index your DataFrame onto that full grid
df = (
    df
    .set_index(['Country','Year'])
    .reindex(full_idx)
    .reset_index()
)

# 4) Bring ISO3 back (it’s constant within each Country)
#    you can create a mapping before the reindex step:
iso_map = unicorn_startups_country_df.drop_duplicates('Country').set_index('Country')['ISO3']
df['ISO3'] = df['Country'].map(iso_map)

# 5) Fill “flow” columns with zeros
df['N_Unicorns'] = df['N_Unicorns'].fillna(0)
df['Total_Val']  = df['Total_Val'].fillna(0.0)
df["Startups"] = df["Startups"].apply(lambda x: x if isinstance(x, dict) else {})
df["Industries"] = df["Industries"].apply(lambda x: x if isinstance(x, list) else [])
#df["cities"] = df["cities"].apply(lambda x: x if isinstance(x, list) else [])

# 6) Sort, then recompute your cumulative sums by country
df = df.sort_values(['Country','Year'])
df['N_Unicorns_Cumulative'] = df.groupby('Country')['N_Unicorns'].cumsum()
df['Total_Val_Cumulative']  = df.groupby('Country')['Total_Val'].cumsum()
df["Startups_Cumulative"] = df.groupby("Country")["Startups"].apply(lambda s: list(accumulate(s, lambda d, new: {**d, **new}))).explode().values
df["Industries_Cumulative"] = df.groupby("Country")["Industries"].apply(lambda s: list(accumulate(s, operator.add))) .explode().values
#df["cities_cumulative"] = df.groupby("Country")["cities"].apply(lambda s: list(accumulate(s, operator.add))).explode().values

unicorn_startups_country_df = df.reset_index(drop=True)
unicorn_startups_country_df

,Country,Year,ISO3,N_Unicorns,Total_Val,Startups,Industries,N_Unicorns_Cumulative,Total_Val_Cumulative,Startups_Cumulative,Industries_Cumulative
0,Argentina,2015,ARG,0.0,0.00,{},[],0.0,0.00,{},[]
1,Argentina,2016,ARG,0.0,0.00,{},[],0.0,0.00,{},[]
2,Argentina,2017,ARG,0.0,0.00,{},[],0.0,0.00,{},[]
3,Argentina,2018,ARG,0.0,0.00,{},[],0.0,0.00,{},[]
4,Argentina,2019,ARG,0.0,0.00,{},[],0.0,0.00,{},[]
...,...,...,...,...,...,...,...,...,...,...,...
219,United States,2017,USA,12.0,48.27,"{'JUUL Labs': 12.0, 'reddit': 10.0, 'Niantic':...","[Consumer & Retail, Internet Software & Servic...",30.0,99.44,"{'Gusto': 10.0, 'Tanium': 9.0, 'Zenefits': 4.5...","[Fintech, Cybersecurity, Fintech, E-commerce &..."
220,United States,2018,USA,42.0,222.82,"{'Epic Games': 28.7, 'Plaid Technologies': 13....","[Other, Fintech, Health, Fintech, Health, Inte...",72.0,322.26,"{'Gusto': 10.0, 'Tanium': 9.0, 'Zenefits': 4.5...","[Fintech, Cybersecurity, Fintech, E-commerce &..."
221,United States,2019,USA,48.0,231.00,"{'Databricks': 38.0, 'Chime': 25.0, 'Grammarly...","[Data management & Analytics, Fintech, Interne...",120.0,553.26,"{'Gusto': 10.0, 'Tanium': 9.0, 'Zenefits': 4.5...","[Fintech, Cybersecurity, Fintech, E-commerce &..."
222,United States,2020,USA,64.0,224.53,"{'goPuff': 15.0, 'Figma': 10.0, 'Notion Labs':...","[E-commerce & Direct-To-Consumer, Internet Sof...",184.0,777.79,"{'Gusto': 10.0, 'Tanium': 9.0, 'Zenefits': 4.5...","[Fintech, Cybersecurity, Fintech, E-commerce &..."


## Aggregate data for unicorn startupts per (Industry, Country)

In [27]:
unicorn_startups_df.head()

,Company,Valuation,Year,Country,City,Industry,ISO3
0,Gusto,10.00,2015,United States,San Francisco,Fintech,USA
1,Tanium,9.00,2015,United States,Kirkland,Cybersecurity,USA
2,Zenefits,4.50,2015,United States,San Francisco,Fintech,USA
3,Thumbtack,3.20,2015,United States,San Francisco,E-commerce & Direct-To-Consumer,USA
4,Illumio,2.75,2015,United States,Sunnyvale,Cybersecurity,USA


In [28]:
unicorn_startups_industry_df = unicorn_startups_df.groupby(["Industry", "Country", "ISO3", "Year"], as_index=False).agg(N_Unicorns = ("ISO3", "size"), 
                                                                                                   Total_Val = ("Valuation", "sum"),
                                                                                                   Startups = ("Valuation",
                                                                                                               lambda s: dict(zip(unicorn_startups_df.loc[s.index, "Company"], s))), 
                                                                                                   #cities = ("City", list)
                                                                                                  ).sort_values(by=["Industry", "Country", "Year"])
unicorn_startups_industry_df[["N_Unicorns_Cumulative", "Total_Val_Cumulative"]] = unicorn_startups_industry_df.groupby(['Industry', "Country"]).agg(N_Unicorns = ("N_Unicorns", "cumsum"), Total_Val = ("Total_Val", "cumsum"))
unicorn_startups_industry_df = unicorn_startups_industry_df.sort_values(by=["Year", "Industry"]).reset_index(drop=True)
unicorn_startups_industry_df.head()

,Industry,Country,ISO3,Year,N_Unicorns,Total_Val,Startups,N_Unicorns_Cumulative,Total_Val_Cumulative
0,Artificial Intelligence,United States,USA,2015,1,2.30,{'Uptake': 2.3},1,2.30
1,Auto & Transportation,France,FRA,2015,1,2.00,{'BlaBlaCar': 2.0},1,2.00
2,Cybersecurity,United States,USA,2015,2,11.75,"{'Tanium': 9.0, 'Illumio': 2.75}",2,11.75
3,E-commerce & Direct-To-Consumer,United States,USA,2015,2,4.30,"{'Thumbtack': 3.2, 'AppDirect': 1.1}",2,4.30
4,Fintech,United States,USA,2015,2,14.50,"{'Gusto': 10.0, 'Zenefits': 4.5}",2,14.50


### Fill missing data in Unicorn Startups data

In [29]:
df = unicorn_startups_industry_df.copy()

# 1) Determine the full list of years and industries
years     = df['Year'].sort_values().unique()
industries = df['Industry'].unique()
countries = df["Country"].unique()

# 2) Build a full MultiIndex of Industry × Year
full_idx = pd.MultiIndex.from_product(
    [industries, years, countries],
    names=['Industry','Year', "Country"]
)

# 3) Re‑index your DataFrame onto that full grid
df = (
    df
    .set_index(['Industry','Year', "Country"])
    .reindex(full_idx)
    .reset_index()
)

df['N_Unicorns'] = df['N_Unicorns'].fillna(0)
df['Total_Val']  = df['Total_Val'].fillna(0.0)
df["Startups"] = df["Startups"].apply(lambda x: x if isinstance(x, dict) else {})
df['ISO3'] = df['Country'].apply(country_to_iso3)

# 6) Sort, then recompute your cumulative sums by country
df = df.sort_values(['Industry','Year', "Country"])
df['N_Unicorns_Cumulative'] = df.groupby(['Industry', "Country"])['N_Unicorns'].cumsum()
df['Total_Val_Cumulative']  = df.groupby(['Industry', "Country"])['Total_Val'].cumsum()
df["Startups_Cumulative"] = df.groupby(['Industry', "Country"])["Startups"].apply(lambda s: list(accumulate(s, lambda d, new: {**d, **new}))).explode().values

unicorn_startups_industry_df = df.reset_index(drop=True)
unicorn_startups_industry_df

,Industry,Year,Country,ISO3,N_Unicorns,Total_Val,Startups,N_Unicorns_Cumulative,Total_Val_Cumulative,Startups_Cumulative
0,Artificial Intelligence,2015,Argentina,ARG,0.0,0.0,{},0.0,0.00,{}
1,Artificial Intelligence,2015,Australia,AUS,0.0,0.0,{},0.0,0.00,{}
2,Artificial Intelligence,2015,Austria,AUT,0.0,0.0,{},0.0,0.00,{}
3,Artificial Intelligence,2015,Belgium,BEL,0.0,0.0,{},0.0,0.00,{}
4,Artificial Intelligence,2015,Brazil,BRA,0.0,0.0,{},0.0,0.00,{}
...,...,...,...,...,...,...,...,...,...,...
3355,Travel,2021,Sweden,SWE,0.0,0.0,{},0.0,0.00,{}
3356,Travel,2021,Switzerland,CHE,0.0,0.0,{},0.0,0.00,{'TripActions': 7.25}
3357,Travel,2021,Thailand,THA,0.0,0.0,{},0.0,0.00,"{'TripActions': 7.25, 'Sonder': 1.3}"
3358,Travel,2021,United Arab Emirates,ARE,0.0,0.0,{},0.0,0.00,"{'TripActions': 7.25, 'Sonder': 1.3}"


## Merge countries with their geom data

In [30]:
"""world = gpd.read_file("../data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp").to_crs(4326)
countries_gdf = world[['ISO_A3','geometry']].rename(columns={'ISO_A3':'ISO3', "geometry":"Country_Geom"})

def map_geom(x, countries_gdf):
    iso3s_geoms = []
    for iso3s in x:
        try:
            geom = countries_gdf.loc[countries_gdf["ISO3"] == iso3s]["Country_Geom"]
            iso3s_geoms.append([iso3s, geom]) 
        except:    
            iso3s_geoms.append([iso3s, None])
    return iso3s_geoms
unicorn_startups_industry_df["ISO3S"] = unicorn_startups_industry_df["ISO3S"].apply(lambda x: map_geom(x, countries_gdf)) 


unicorn_startups_country_df = pd.merge(unicorn_startups_country_df, countries_gdf, on='ISO3', how='left')
unicorn_startups_country_df = gpd.GeoDataFrame(unicorn_startups_country_df, geometry='Country_Geom').to_crs(4326)
qol_df = pd.merge(qol_df, countries_gdf, on='ISO3', how='left')
qol_df = gpd.GeoDataFrame(qol_df, geometry='Country_Geom').to_crs(4326)

unicorn_startups_industry_df.head()"""


'world = gpd.read_file("../data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp").to_crs(4326)\ncountries_gdf = world[[\'ISO_A3\',\'geometry\']].rename(columns={\'ISO_A3\':\'ISO3\', "geometry":"Country_Geom"})\n\ndef map_geom(x, countries_gdf):\n    iso3s_geoms = []\n    for iso3s in x:\n        try:\n            geom = countries_gdf.loc[countries_gdf["ISO3"] == iso3s]["Country_Geom"]\n            iso3s_geoms.append([iso3s, geom]) \n        except:    \n            iso3s_geoms.append([iso3s, None])\n    return iso3s_geoms\nunicorn_startups_industry_df["ISO3S"] = unicorn_startups_industry_df["ISO3S"].apply(lambda x: map_geom(x, countries_gdf)) \n\n\nunicorn_startups_country_df = pd.merge(unicorn_startups_country_df, countries_gdf, on=\'ISO3\', how=\'left\')\nunicorn_startups_country_df = gpd.GeoDataFrame(unicorn_startups_country_df, geometry=\'Country_Geom\').to_crs(4326)\nqol_df = pd.merge(qol_df, countries_gdf, on=\'ISO3\', how=\'left\')\nqol_df = gpd.GeoDataFrame(qol_df, geom

In [31]:
unicorn_startups_country_df.head()

,Country,Year,ISO3,N_Unicorns,Total_Val,Startups,Industries,N_Unicorns_Cumulative,Total_Val_Cumulative,Startups_Cumulative,Industries_Cumulative
0,Argentina,2015,ARG,0.0,0.0,{},[],0.0,0.0,{},[]
1,Argentina,2016,ARG,0.0,0.0,{},[],0.0,0.0,{},[]
2,Argentina,2017,ARG,0.0,0.0,{},[],0.0,0.0,{},[]
3,Argentina,2018,ARG,0.0,0.0,{},[],0.0,0.0,{},[]
4,Argentina,2019,ARG,0.0,0.0,{},[],0.0,0.0,{},[]


In [32]:
qol_df.head()

,Country,Quality_of_Life_Index,Purchasing_Power_Index,Cost_of_Living_Index,Property_Price_to_Income_Ratio,Year,GDP_Per_Capita,ISO3
0,Argentina,77.0,59.4,67.1,11.6,2015,14833.199680,ARG
1,Australia,180.8,110.4,99.3,7.1,2015,52009.802759,AUS
2,Austria,182.6,104.6,76.9,9.6,2015,43908.420277,AUT
3,Belgium,136.0,86.2,87.2,6.5,2015,40889.673570,BEL
4,Brazil,29.8,41.2,55.3,16.7,2015,8936.195589,BRA


## Save Data in CSV format

In [33]:
filename = "Unicorn_Startups_Per_Industry.csv"
unicorn_startups_industry_df.to_csv(posixpath.join(PREPROCESSED_DATA_PATH, filename), index=False)
unicorn_startups_industry_df = pd.read_csv(posixpath.join(PREPROCESSED_DATA_PATH, filename))
unicorn_startups_industry_df

,Industry,Year,Country,ISO3,N_Unicorns,Total_Val,Startups,N_Unicorns_Cumulative,Total_Val_Cumulative,Startups_Cumulative
0,Artificial Intelligence,2015,Argentina,ARG,0.0,0.0,{},0.0,0.00,{}
1,Artificial Intelligence,2015,Australia,AUS,0.0,0.0,{},0.0,0.00,{}
2,Artificial Intelligence,2015,Austria,AUT,0.0,0.0,{},0.0,0.00,{}
3,Artificial Intelligence,2015,Belgium,BEL,0.0,0.0,{},0.0,0.00,{}
4,Artificial Intelligence,2015,Brazil,BRA,0.0,0.0,{},0.0,0.00,{}
...,...,...,...,...,...,...,...,...,...,...
3355,Travel,2021,Sweden,SWE,0.0,0.0,{},0.0,0.00,{}
3356,Travel,2021,Switzerland,CHE,0.0,0.0,{},0.0,0.00,{'TripActions': 7.25}
3357,Travel,2021,Thailand,THA,0.0,0.0,{},0.0,0.00,"{'TripActions': 7.25, 'Sonder': 1.3}"
3358,Travel,2021,United Arab Emirates,ARE,0.0,0.0,{},0.0,0.00,"{'TripActions': 7.25, 'Sonder': 1.3}"


In [34]:
filename = "Unicorn_Startups_Per_Country.csv"
unicorn_startups_country_df.to_csv(posixpath.join(PREPROCESSED_DATA_PATH, filename), index=False)
unicorn_startups_country_df = pd.read_csv(posixpath.join(PREPROCESSED_DATA_PATH, filename))
unicorn_startups_country_df

,Country,Year,ISO3,N_Unicorns,Total_Val,Startups,Industries,N_Unicorns_Cumulative,Total_Val_Cumulative,Startups_Cumulative,Industries_Cumulative
0,Argentina,2015,ARG,0.0,0.00,{},[],0.0,0.00,{},[]
1,Argentina,2016,ARG,0.0,0.00,{},[],0.0,0.00,{},[]
2,Argentina,2017,ARG,0.0,0.00,{},[],0.0,0.00,{},[]
3,Argentina,2018,ARG,0.0,0.00,{},[],0.0,0.00,{},[]
4,Argentina,2019,ARG,0.0,0.00,{},[],0.0,0.00,{},[]
...,...,...,...,...,...,...,...,...,...,...,...
219,United States,2017,USA,12.0,48.27,"{'JUUL Labs': 12.0, 'reddit': 10.0, 'Niantic':...","['Consumer & Retail', 'Internet Software & Ser...",30.0,99.44,"{'Gusto': 10.0, 'Tanium': 9.0, 'Zenefits': 4.5...","['Fintech', 'Cybersecurity', 'Fintech', 'E-com..."
220,United States,2018,USA,42.0,222.82,"{'Epic Games': 28.7, 'Plaid Technologies': 13....","['Other', 'Fintech', 'Health', 'Fintech', 'Hea...",72.0,322.26,"{'Gusto': 10.0, 'Tanium': 9.0, 'Zenefits': 4.5...","['Fintech', 'Cybersecurity', 'Fintech', 'E-com..."
221,United States,2019,USA,48.0,231.00,"{'Databricks': 38.0, 'Chime': 25.0, 'Grammarly...","['Data management & Analytics', 'Fintech', 'In...",120.0,553.26,"{'Gusto': 10.0, 'Tanium': 9.0, 'Zenefits': 4.5...","['Fintech', 'Cybersecurity', 'Fintech', 'E-com..."
222,United States,2020,USA,64.0,224.53,"{'goPuff': 15.0, 'Figma': 10.0, 'Notion Labs':...","['E-commerce & Direct-To-Consumer', 'Internet ...",184.0,777.79,"{'Gusto': 10.0, 'Tanium': 9.0, 'Zenefits': 4.5...","['Fintech', 'Cybersecurity', 'Fintech', 'E-com..."


In [35]:
filename = "QOL.csv"
qol_df.to_csv(posixpath.join(PREPROCESSED_DATA_PATH, filename), index=False)
qol_df = pd.read_csv(posixpath.join(PREPROCESSED_DATA_PATH, filename))
qol_df

,Country,Quality_of_Life_Index,Purchasing_Power_Index,Cost_of_Living_Index,Property_Price_to_Income_Ratio,Year,GDP_Per_Capita,ISO3
0,Argentina,77.0,59.4,67.1,11.6,2015,14833.199680,ARG
1,Australia,180.8,110.4,99.3,7.1,2015,52009.802759,AUS
2,Austria,182.6,104.6,76.9,9.6,2015,43908.420277,AUT
3,Belgium,136.0,86.2,87.2,6.5,2015,40889.673570,BEL
4,Brazil,29.8,41.2,55.3,16.7,2015,8936.195589,BRA
...,...,...,...,...,...,...,...,...
219,Sweden,171.4,90.6,79.2,8.6,2021,93076.833587,SWE
220,Switzerland,190.8,111.0,131.7,8.4,2021,3484.385958,CHE
221,Thailand,100.3,31.4,49.3,22.2,2021,78270.600992,THA
222,United Arab Emirates,156.0,85.7,61.7,4.9,2021,6830.894725,ARE
